In [ ]:
import os
import sys
import openai
# two directories up
sys.path.append(os.path.join(os.getcwd(), os.pardir, os.pardir))
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from pymolassistant.db.vector_store import load_config, load_vector_store, OPENAI_API_KEY

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

config = load_config()

Qdrant API Key:  f3TPb
OpenAI API Key:  sk-pr


In [2]:
df1 = pd.read_csv('../data/pymol_commands_api.csv')
df2 = pd.read_csv('../data/pymol_commands_wiki_1.csv')
df3 = pd.read_csv('../data/pymol_commands_wiki_2.csv')

In [3]:
df1['name'] = df1['name'].str.split().str.get(0)

In [4]:
wiki = pd.concat([df2, df3], ignore_index=True)

In [5]:
wiki = wiki.fillna('')

In [6]:
# just two columns: 'name', and 'content' (the concat of all columns except 'name')
wiki['content'] = wiki.apply(lambda x: ' '.join(x.dropna().values), axis=1)
wiki = wiki[['name', 'content']]
wiki

,name,content
0,Align,Align Two proteins after structure alignment\...
1,Alter,Alter The iterate command executes a Python ex...
2,Alter State,Alter State The iterate command executes a Pyt...
3,Attach,Attach attach adds a single atom onto the pick...
4,Backward,Backward backward moves the movie back one fra...
...,...,...
198,Volume,Volume Volume visualization of electron densi...
199,Volume color,Volume color volume_color set or get the volum...
200,Window,Window window controls the visibility of PyMOL...
201,Wizard,Wizard wizard launches one of the built-in wiz...


In [7]:
openai_client = openai.Client()

In [8]:
#wiki['vector_name'] = wiki['name'].progress_apply(lambda x: openai_client.embeddings.create(input=x, model="text-embedding-3-small"))

100%|██████████| 203/203 [01:21<00:00,  2.48it/s]


In [10]:
#wiki['vector_content'] = wiki['content'].progress_apply(lambda x: openai_client.embeddings.create(input=x, model="text-embedding-3-small"))

100%|██████████| 203/203 [01:09<00:00,  2.91it/s]


In [25]:
# store directly the embeddings instead of the CreateEmbeddingResponse
wiki['vector_name'] = wiki['vector_name'].apply(lambda x: x.data)
wiki['vector_content'] = wiki['vector_content'].apply(lambda x: x.data)

In [40]:
wiki['vector_name'] = wiki['vector_name'].apply(lambda x: x[0].to_dict())
wiki['vector_content'] = wiki['vector_content'].apply(lambda x: x[0].to_dict())

Control usage/cost: 
- https://platform.openai.com/usage

In [41]:
#wiki.to_parquet(config.get('data').get("parquet_file"))

import pickle

with open('../../prep/data/pymol-docs.pkl', 'wb') as f:
    pickle.dump(wiki, f)

In [42]:
if not os.path.exists('../../prep/data/pymol-docs.pkl'):
    raise FileNotFoundError("Parquet file not found in the data directory.")

# Load the data from the pkl
with open('../../prep/data/pymol-docs.pkl', 'rb') as f:
    pymol_docs = pickle.load(f)

In [43]:
pymol_docs.vector_name.iloc[0]

{'embedding': [-0.030813321471214294,
  0.022618014365434647,
  0.02874183841049671,
  0.04632355645298958,
  0.011542048305273056,
  -0.004803252872079611,
  -0.05300408974289894,
  -0.05339249595999718,
  -0.028690051287412643,
  0.013005034066736698,
  0.004443980287760496,
  0.029648112133145332,
  0.038400132209062576,
  0.0002969666093122214,
  0.020611263811588287,
  0.004039393272250891,
  -0.03578488156199455,
  -0.029052559286355972,
  0.04746287316083908,
  0.04163682460784912,
  -0.001014703419059515,
  0.00989780854433775,
  0.033247314393520355,
  0.014888789504766464,
  -0.025297995656728745,
  -0.014668694697320461,
  0.015134778805077076,
  0.020378222689032555,
  0.048239678144454956,
  -0.027473052963614464,
  0.04075644537806511,
  -0.03469735383987427,
  0.047100361436605453,
  -0.04065287113189697,
  0.03635454177856445,
  0.039073362946510315,
  0.017076794058084488,
  0.012888513505458832,
  -0.01945900171995163,
  0.014370919205248356,
  -0.02194478176534176,
 

In [35]:
dir(pymol_docs.vector_name.iloc[0][0])

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_a

In [39]:
pymol_docs.vector_name.iloc[0][0].to_dict()

{'embedding': [-0.030813321471214294,
  0.022618014365434647,
  0.02874183841049671,
  0.04632355645298958,
  0.011542048305273056,
  -0.004803252872079611,
  -0.05300408974289894,
  -0.05339249595999718,
  -0.028690051287412643,
  0.013005034066736698,
  0.004443980287760496,
  0.029648112133145332,
  0.038400132209062576,
  0.0002969666093122214,
  0.020611263811588287,
  0.004039393272250891,
  -0.03578488156199455,
  -0.029052559286355972,
  0.04746287316083908,
  0.04163682460784912,
  -0.001014703419059515,
  0.00989780854433775,
  0.033247314393520355,
  0.014888789504766464,
  -0.025297995656728745,
  -0.014668694697320461,
  0.015134778805077076,
  0.020378222689032555,
  0.048239678144454956,
  -0.027473052963614464,
  0.04075644537806511,
  -0.03469735383987427,
  0.047100361436605453,
  -0.04065287113189697,
  0.03635454177856445,
  0.039073362946510315,
  0.017076794058084488,
  0.012888513505458832,
  -0.01945900171995163,
  0.014370919205248356,
  -0.02194478176534176,
 